In [6]:
import pandas as pd
import nltk
import re
import numpy as np
from textblob import TextBlob
import matplotlib.pyplot as plt
from sklearn.decomposition import LatentDirichletAllocation
from collections import Counter
from nltk.sentiment import SentimentIntensityAnalyzer
import seaborn as sns
#from wordcloud import WordCloud, STOPWORDS

In [7]:
data = pd.read_excel('/Users/student/Desktop/chatgpt1.xlsx')
print(data)
data.head

                        Datetime             Tweet Id  \
0      2023-01-22 13:44:34+00:00  1617156270871689984   
1      2023-01-22 13:44:39+00:00  1617156291046129920   
2      2023-01-22 13:44:44+00:00  1617156308926340096   
3      2023-01-22 13:44:49+00:00  1617156332297250048   
4      2023-01-22 13:44:52+00:00  1617156345064570112   
...                          ...                  ...   
49996  2023-01-24 06:57:56+00:00  1617778712082089984   
49997  2023-01-24 06:57:59+00:00  1617778726393240064   
49998  2023-01-24 06:58:00+00:00  1617778728481989888   
49999  2023-01-24 06:58:01+00:00  1617778731678040064   
50000  2023-01-24 06:58:01+00:00  1617778733355790080   

                                                    Text         Username  \
0      ChatGPTã§éŠã¶ã®å¿˜ã‚Œã¦ãŸï¼ï¼\næ›¸é¡žä...      mochico0123   
1      @AlexandrovnaIng Prohibition of ChatGPT has be...  Caput_LupinumSG   
2      Schaut Euch an, was @fobizz @DianaKnodel alles...            ciffi   
3      

<bound method NDFrame.head of                         Datetime             Tweet Id  \
0      2023-01-22 13:44:34+00:00  1617156270871689984   
1      2023-01-22 13:44:39+00:00  1617156291046129920   
2      2023-01-22 13:44:44+00:00  1617156308926340096   
3      2023-01-22 13:44:49+00:00  1617156332297250048   
4      2023-01-22 13:44:52+00:00  1617156345064570112   
...                          ...                  ...   
49996  2023-01-24 06:57:56+00:00  1617778712082089984   
49997  2023-01-24 06:57:59+00:00  1617778726393240064   
49998  2023-01-24 06:58:00+00:00  1617778728481989888   
49999  2023-01-24 06:58:01+00:00  1617778731678040064   
50000  2023-01-24 06:58:01+00:00  1617778733355790080   

                                                    Text         Username  \
0      ChatGPTã§éŠã¶ã®å¿˜ã‚Œã¦ãŸï¼ï¼\næ›¸é¡žä...      mochico0123   
1      @AlexandrovnaIng Prohibition of ChatGPT has be...  Caput_LupinumSG   
2      Schaut Euch an, was @fobizz @DianaKnodel alles.

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50001 entries, 0 to 50000
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Datetime        50001 non-null  object
 1   Tweet Id        50001 non-null  int64 
 2   Text            49996 non-null  object
 3   Username        50001 non-null  object
 4   Permalink       50001 non-null  object
 5   User            50001 non-null  object
 6   Outlinks        19942 non-null  object
 7   CountLinks      19942 non-null  object
 8   ReplyCount      50001 non-null  int64 
 9   RetweetCount    50001 non-null  int64 
 10  LikeCount       50001 non-null  int64 
 11  QuoteCount      50001 non-null  int64 
 12  ConversationId  50001 non-null  int64 
 13  Language        50001 non-null  object
 14  Source          50001 non-null  object
 15  Media           9502 non-null   object
 16  QuotedTweet     3563 non-null   object
 17  MentionedUsers  17169 non-null  object
 18  hashta

In [22]:
#Creating a function that takes care of all the preprocessing stuff.
def preprocess():

  data['Text'] = data['Text'].str.lower() # Ensuring all words in the Tweet column of training data are lowercased

  # Parsing the stop_words.txt file and storing all the words in a list.
  stopwords = []
  with open('Downloads/sample.txt','r') as file:    
      for line in file:         
          for word in line.split():            
              stopwords.append(word)

  # Removing all stopwords from all the tweets in training data.
  #df["Text"] = df["Text"].apply(lambda func: ' '.join(sw 
   #                                         for sw in func.split() 
    #                                        if sw not in stopwords))
  #Training Data
  data['Text'] = data['Text'].str.replace(r'http?://[^\s<>"]+|www\.[^\s<>"]+', '') # Removing hyperlinks from all the tweets
  data['Text'] = data['Text'].str.replace('@[A-Za-z0-9]+', '') # Removing usernames from all the tweets.
  data['Text'] = data['Text'].str.replace(r'\B#\w*[a-zA-Z]+\w*', '') # Removing hashtags, including the text, from all the tweets
  data['Text'] = data['Text'].str.replace('\d+', '') # Removing numbers from all the tweets

  special_chars = ["!",'"',"%","&","amp","'","(",")", "*","+",",","-",".",
                  "/",":",";","<","=",">","?","[","\\","]","^","_",
                  "`","{","|","}","~","–","@","#","$"]

  for c in special_chars:
      data['Text'] = data['Text'].str.replace(c,'') # Removing all special characters from all the tweets

In [23]:
preprocess()

FileNotFoundError: [Errno 2] No such file or directory: 'Downloads/sample.txt'

In [13]:
#Feature and Label selection

y_col = 'Language'
y = data[y_col]

x_col = 'Text'
x = data[x_col]
y, x

(0        ja
 1        en
 2        de
 3        en
 4        tr
          ..
 49996    de
 49997    de
 49998    it
 49999    en
 50000    es
 Name: Language, Length: 50001, dtype: object,
 0        ChatGPTã§éŠã¶ã®å¿˜ã‚Œã¦ãŸï¼ï¼\næ›¸é¡žä...
 1        @AlexandrovnaIng Prohibition of ChatGPT has be...
 2        Schaut Euch an, was @fobizz @DianaKnodel alles...
 3        Bow down to chatGPT ðŸ«¡..... https://t.co/ENT...
 4        Profilinde vatan, TÃ¼rkiye falan yazan bireyle...
                                ...                        
 49996    #ChatGPT ist ein #Chatbot, der durch kÃ¼nstlic...
 49997    @r8r Ich hab mal die AI dazu befragt (ChatGPT)...
 49998    5 minuti di #chatGPT e ho capito che apprende ...
 49999    Portland Shop Uses ChatGPT To Tell Family Stor...
 50000    Ahora sueÃ±o con el dÃ­a en que Amazon integre...
 Name: Text, Length: 50001, dtype: object)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

# Assuming training_data_x is a pandas Series or DataFrame column containing the text data

training_data_x = training_data_x.fillna('')
testing_data_x = testing_data_x.fillna('')
training_data_y = training_data_y.fillna('')
testing_data_y = testing_data_y.fillna('')

#Vectorization
X_train_tfidf = tfidf.fit_transform(training_data_x)
X_test_tfidf = tfidf.transform(testing_data_x)


NameError: name 'training_data_x' is not defined

In [15]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [16]:
knn = KNeighborsClassifier(n_neighbors=3)  # You can adjust the value of n_neighbors as per your requirements
knn.fit(X_train_tfidf, training_data_y)
y_pred = knn.predict(X_test_tfidf)

# Evaluate the accuracy of the model
accuracy = accuracy_score(testing_data_y, y_pred)


NameError: name 'X_train_tfidf' is not defined

In [17]:
accuracy

NameError: name 'accuracy' is not defined